# Logistique de la vaccination Covid en Belgique
### Groupe 41
**Aigret Julien 8343-13-00,  
Suetens Lucas,  
Zaleschi Florina**
## Partie 1

In [30]:
positive_inf=float('inf')
negative_inf=-float('inf')

nb_days = 350
deliveries = [[7, 80000],
[14, 80000],
[21, 60000],
[28, 60000],
[35, 40000],
[42, 40000],
[49, 40000],
[56, 40000],
[63, 60000],
[70, 60000],
[77, 60000],
[84, 60000],
[91, 80000],
[98, 80000],
[105, 80000],
[112, 80000],
[119, 100000],
[126, 100000],
[133, 100000],
[140, 100000],
[147, 100000],
[154, 100000],
[161, 100000],
[168, 100000],
[175, 120000],
[182, 120000],
[189, 120000],
[196, 120000],
[203, 120000],
[210, 120000],
[217, 120000],
[224, 120000],
[231, 120000],
[238, 120000],
[245, 120000],
[252, 120000],
[259, 150000],
[266, 150000],
[273, 150000],
[280, 150000],
[287, 150000],
[294, 150000],
[301, 150000],
[308, 150000],
[315, 150000],
[322, 150000],
[329, 150000],
[336, 150000],
[343, 150000],
[350, 150000]]

population = {'Y': 3778123, 'A': 2846993, 'S': 2790883, 'O': 1390502, 'C': 111533}

def frac_to_get_ill(classe, jour):
    import math
    if classe == 'Y':
        return 0.000298 * (1/5 + math.sin(jour/50-1/5)**2)
    if classe == 'A':
        return 0.000301 * (1/5 + math.sin(jour/50)**2)
    if classe == 'S':
        return 0.000204 * (1/5 + math.sin(jour/50-1/5)**2)
    if classe == 'O':
        return 0.000209 * (1/5 + math.sin(jour/50-2/5)**2)
    if classe == 'C':
        return 0.000329 * (1/5 + math.sin(jour/50-2/5)**2)

def frac_to_die(classe, jour):
    import math
    if classe == 'Y':
        return 0.000100 * (6/5-jour/1000)
    if classe == 'A':
        return 0.000400 * (6/5-jour/1000)
    if classe == 'S':
        return 0.005790 * (6/5-jour/1000)
    if classe == 'O':
        return 0.027179 * (6/5-jour/1000)
    if classe == 'C':
        return 0.150000 * (6/5-jour/1000)
    
to_be_vaccined =  {'Y' : 0.3, 'A' : 0.6, 'S' : 0.7, 'O' : 0.9, 'C' : 0.9}
transport_cost = [0]
max_delivery = [positive_inf]
vaccination_cost = [15]
max_vaccination = [14646]
stock_capa = [0]
budget= 100 * 10**6
classes = ['Y', 'A', 'S', 'O', 'C']

### Question I.1

In [103]:
"""
Variables :
- susc['classe'][t] : population de la classe 'classe' susceptible de tomber malade (dictionnaire) au jour t
- mort['classe'][t] : population de la classe 'classe' morte (dictionnaire) au jour t
fonction obj : min sum(d['classes'][350])
contraintes :
au jour t, on vaccine 
"""
#import sys
#!{sys.executable} -m pip install mip
from mip import *

m = Model()

susc = dict()
mort = dict()
vaccin_delivered = dict()
vaccine_given = dict()

for C in classes:
    susc[C] = [m.add_var(lb=0) for _ in range(nb_days+1)]
    mort[C] = [m.add_var(lb=0) for _ in range(nb_days+1)]
    vaccine_given[C] = [m.add_var(lb=0) for _ in range(nb_days+1)]

vaccin_delivered = [m.add_var(lb=0) for _ in range(nb_days+1)]

m.objective = minimize(xsum(mort[C][350] for C in classes))

for C in classes:
    m += susc[C][0] == population[C]
    m += mort[C][0] == 0

livraisons = [0 for _ in range(nb_days+1)]
for [day, doses] in deliveries:
    livraisons[day] = doses

for t in range(nb_days):
    m += vaccin_delivered[t+1] <= livraisons[t]
    
for t in range(nb_days+1):
    m += vaccin_delivered[t] <= max_delivery[0]
    m += xsum(vaccine_given[C][t] for C in classes) <= vaccin_delivered[t] #""" + stock, pas pour cette question"""
    m += xsum(vaccine_given[C][t] for C in classes) <= max_vaccination[0]
    
for C in classes:
    for t in range(nb_days):
        m += vaccine_given[C][t] <= susc[C][t] * to_be_vaccined[C]
        m += mort[C][t+1] == mort[C][t] + susc[C][t] * frac_to_get_ill(C,t) * frac_to_die(C,t)
        m += susc[C][t+1] == susc[C][t] - susc[C][t]*frac_to_get_ill(C,t) - vaccine_given[C][t]

# m += vaccine_given['Y'][323] == 0
# m += vaccine_given['Y'][330] == 0
# m += vaccine_given['Y'][337] == 0
# m += vaccine_given['Y'][344] == 0

m += xsum(vaccine_given[C][t] for t in range(nb_days+1) for C in classes)*15 <= budget

status = m.optimize()
if status == OptimizationStatus.OPTIMAL:
    print('optimal solution cost {} found'.format(m.objective_value))
elif status == OptimizationStatus.FEASIBLE:
    print('sol.cost {} found, best possible: {}'.format(m.objective_value, m.objective_bound))
elif status == OptimizationStatus.NO_SOLUTION_FOUND:
    print('no feasible solution found, lower bound is: {}'.format(m.objective_bound))

optimal solution cost 2392.7372478409043 found


In [104]:
for days in range(nb_days):
    for C in classes:
        if vaccine_given[C][days+1].x != 0:
            print(f"Jour {days+1}, classe {C} : {vaccine_given[C][days+1].x}")
print(sum(vaccine_given[C][t].x for t in range(nb_days+1) for C in classes))
for C in classes:
    print(f"Classe {C} : susceptible {susc[C][350].x}, morts {mort[C][350].x}")

2392.7372478409043
Jour 8, classe C : 14646.000000000002
Jour 15, classe C : 14646.0
Jour 22, classe C : 14645.999999999998
Jour 29, classe C : 14646.0
Jour 36, classe C : 14646.0
Jour 43, classe C : 14646.0
Jour 50, classe O : 6751.188380755475
Jour 50, classe C : 7894.811619244524
Jour 57, classe O : 13858.777495456188
Jour 57, classe C : 787.2225045438126
Jour 64, classe O : 14567.559586702288
Jour 64, classe C : 78.4404132977134
Jour 71, classe O : 14638.18974207226
Jour 71, classe C : 7.810257927739544
Jour 78, classe O : 14645.222867301229
Jour 78, classe C : 0.7771326987702979
Jour 85, classe O : 14645.922718859756
Jour 85, classe C : 0.07728114024405597
Jour 92, classe O : 14645.992318177921
Jour 92, classe C : 0.007681822077885109
Jour 99, classe O : 14645.999236611073
Jour 99, classe C : 0.0007633889251099334
Jour 106, classe O : 14645.999924141299
Jour 106, classe C : 7.585869807278369e-05
Jour 113, classe O : 14645.999992460691
Jour 113, classe C : 7.539309852276097e-06
Jou

## Question I.2